# Compute metrics on Gnann data

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from pathlib import Path

import pandas as pd

In [4]:
PROCESSED_DATA_FOLDER_PATH = Path("../data/processed")

DOMAINS_COLUMN = "domain"

MODEL = "pcr-globwb"

INPUTS_COLUMNS = ["pr", "netrad"]
OUTPUTS_COLUMNS = ["evap", "potevap", "qr", "qtot"]

## Load Gnann data

In [5]:
data_df = pd.read_csv(PROCESSED_DATA_FOLDER_PATH.joinpath(f"gnann_data_{MODEL}.csv"), index_col=False)
data_df.head()

,lat,lon,pr,netrad,evap,potevap,qr,qtot,domain
0,82.75,-82.25,116.31531,115.680597,0.00000,0.00000,0.000000,0.000000,wet cold
1,82.75,-81.75,183.33421,109.368197,77.23199,186.63345,78.122990,100.953619,wet cold
2,82.75,-81.25,185.47101,163.255589,77.44580,188.38412,86.587296,103.959123,wet cold
3,82.75,-80.75,186.57018,133.387732,79.07652,183.37039,74.703705,103.137103,wet cold
4,82.75,-80.25,188.50586,107.377043,80.29123,173.71771,69.313614,103.735826,wet cold


## 1 to 1 metrics

In [16]:
from scipy import stats
from sklearn.feature_selection import mutual_info_regression

### 

In [23]:
stats.pearsonr(data_df["pr"], data_df["potevap"])

PearsonRResult(statistic=0.3214771435567031, pvalue=0.0)

In [15]:
stats.spearmanr(data_df["pr"], data_df["potevap"])

SignificanceResult(statistic=0.23459031650794795, pvalue=0.0)

In [19]:
stats.linregress(data_df["pr"], data_df["potevap"])

LinregressResult(slope=0.20451924701725144, intercept=661.069560471054, rvalue=0.3214771435567032, pvalue=0.0, stderr=0.0023945191385510755, intercept_stderr=2.541334602836101)

In [21]:
mutual_info_regression(data_df["pr"].values.reshape(-1, 1),
                       data_df["potevap"].values)

array([0.40136157])

In [ ]:
from typing import List


def compute_scores(df: pd.DataFrame,
                   input_cols: List[str],
                   output_cols: List[str]) -> pd.DataFrame:
    """_summary_

    Args:
        df (pd.DataFrame): _description_
        input_cols (List[str]): _description_
        output_cols (List[str]): _description_

    Returns:
        pd.DataFrame: The different scores for the different combinations of inputs-outputs
    """


    
    return